# How to enable/disable collisions

In [1]:
import sys
sys.path.append('../build')
import libry as ry
import numpy as np
import time

**ry-c++-log** /home/jung-su/git/robotics-course/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback



In [2]:
C = ry.Config()
C.addFile("../scenarios/pandasTable.g")
D = C.view()
# V = ry.ConfigurationViewer()
# V.setConfiguration(C)

In [3]:
#no collisions so far:
C.computeCollisions() #this calls broadphase collision detection
C.getCollisions()     #this reports collisions (in algorithms, use features instead)

[]

In [4]:
#bring into collision:
q = C.getJointState()

for t in range(10):
    [y,J] = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "L_gripperCenter"])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y);

    q = q + .8*vel
    C.setJointState(q)

In [5]:
C.computeCollisions() #this calls broadphase collision detection
C.getCollisions(belowMargin=1.) 

[('L_panda_coll7', 'R_frame', 0.04384857882061481),
 ('L_frame', 'R_frame', 0.062271336259521554),
 ('L_panda_coll7', 'R_frame', -0.0165056236477103),
 ('L_gripper', 'R_frame', 0.061638234421454136),
 ('L_finger1', 'R_frame', 0.08104513748692069),
 ('L_panda_coll7', 'R_panda_coll7', 0.008790855718814328),
 ('L_gripper', 'R_panda_coll7', -0.010603028742971136),
 ('L_finger1', 'R_panda_coll7', -0.0027839621081884175),
 ('L_frame', 'R_gripper', 0.09081440563557787),
 ('L_panda_coll7', 'R_gripper', -0.0039863116105960435),
 ('L_gripper', 'R_gripper', -0.02),
 ('L_finger1', 'R_gripper', -0.008350122088338059),
 ('L_finger2', 'R_gripper', 0.07321833887206175),
 ('L_gripper', 'R_finger1', 0.06962644342495132),
 ('L_finger1', 'R_finger1', 0.03500466193618268),
 ('L_finger2', 'R_finger1', 0.059582137007127306),
 ('L_frame', 'R_finger2', 0.07910327106911726),
 ('L_panda_coll7', 'R_finger2', 0.0068432225174079354),
 ('L_gripper', 'R_finger2', 0.0006805808021984085),
 ('L_finger1', 'R_finger2', 0.

In [6]:
#those are the real penetrations:
C.getCollisions(belowMargin=0.) 

[('L_panda_coll7', 'R_frame', -0.0165056236477103),
 ('L_gripper', 'R_panda_coll7', -0.010603028742971136),
 ('L_finger1', 'R_panda_coll7', -0.0027839621081884175),
 ('L_panda_coll7', 'R_gripper', -0.0039863116105960435),
 ('L_gripper', 'R_gripper', -0.02),
 ('L_finger1', 'R_gripper', -0.008350122088338059)]

In [7]:
#deactivate a collision flag!
f = C.getFrame("R_gripper")
f.setContact(0)

In [8]:
C.computeCollisions()
C.getCollisions(belowMargin=0.) 

[('L_panda_coll7', 'R_frame', -0.0165056236477103),
 ('L_gripper', 'R_panda_coll7', -0.010603028742971136),
 ('L_finger1', 'R_panda_coll7', -0.0027839621081884175)]

In [9]:
#Note: this is exactly the sum of the above:
C.evalFeature(ry.FS.accumulatedCollisions, [])

(array([0.02989261]),
 array([[-8.70327325e-01, -8.44179283e-01, -8.64238152e-01,
          1.21674891e+00, -5.35130235e-02,  3.94913772e-01,
         -4.67555907e-02,  1.09051673e+00,  1.07776584e+00,
          1.03000899e+00, -7.89139561e-01, -7.41472816e-02,
         -1.80039389e-01,  4.28506360e-18]]))

Notes:
* The contact flag of frames is an integer: 0 = does not contribute to contact lists, 1 = always contributes, negative = only contributes to collision pairs that are topologically distant in the kinematic tree
* WARNING: the broad phase collision engine is created at first call - You cannot enable collisions between objects that were disabled during first call of the collision engine
* TODO: enable recreation of a fresh collision engine for a changed configuration

# How to attach frames - faking grasps
Note, this is not real grasping. Just editing the kinematic tree in your configuration

In [ ]:
import os
os._exit(0)

In [1]:
import sys
sys.path.append('../build')
import libry as ry
import numpy as np
import time

**ry-c++-log** /home/jung-su/git/robotics-course/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback



In [2]:
C = ry.Config()
C.addFile("../scenarios/pandasTable.g")
D = C.view()

In [3]:
C.attach("L_gripper", "R_gripper")

In [4]:
#move a bit around

q = C.getJointState()

for t in range(30):
    q[0] = np.sin(t/10)
    
    C.setJointState(q)
    time.sleep(0.1)